In [1]:
#!/usr/bin/env python3
import os, sys, signal,rospy, argparse, csv

from tqdm import tqdm
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.spatial.distance import cdist
import copy, pickle
import open3d as o3d
import open3d.visualization as vis
import torch
from torch.autograd import Variable

from open3d_ros_helper import open3d_ros_helper as orh
from geometry_msgs.msg import Pose, PoseArray, Point # PoseArray, Pose

from models.mdgat_denseKITTI import MDGAT

torch.set_grad_enabled(True)
torch.multiprocessing.set_sharing_strategy('file_system')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
parser = argparse.ArgumentParser(
    description='Point cloud matching and pose evaluation',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)

opt = argparse.Namespace(
    dataset_dir = '/media/vision/Seagate/DataSets/denseKITTI',
    # slam_dir = '',
    data_folder = 'harris_3D',
    local_global = False,
    seq_num = 0,
    visualize = False,
    vis_line_width = 0.2,
    calculate_pose = True,
    learning_rate = 0.0001,
    batch_size = 1,
    train_path = './denseKITTI/',
    model_out_path = './models/checkpoint',
    memory_is_enough = True,
    local_rank = 0,
    txt_path = './KITTI/preprocess-random-full',
    keypoints_path = './denseKITTI/keypoints',
    resume_model = './checkpoint/denseKITTI/mdgat-l9-gap_loss-pointnet-06_07_15_02/train_step3/nomutualcheck-mdgat-batch32-lr0.0001-gap_loss-pointnet-Hariss3D-06_07_15_02/model_epoch_155.pth',
    loss_method = 'gap_loss',
    net = 'mdgat',
    mutual_check = False,
    k = [64, None, 64, None, 32, None, 32, None],
    l = 9,
    descriptor = 'pointnet',
    keypoints = 'harris_3D',
    ensure_kpts_num = False,
    max_keypoints = -1,
    match_threshold = 0.2,
    threshold = 0.5,
    triplet_loss_gamma = 0.5,
    sinkhorn_iterations = 100,
    train_step = 3,

    fpfh_normal_radiuse = 0.3,
    fpfh_descriptors_radiuse = 1.0,
    seq_list = [0],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    transform_opt = 0
)

In [3]:
class Kitti_Rops_dataset_loader():
    def __init__(self, args, seq_num) -> None:
        self.dataset_dir = args.dataset_dir
        self.seq = seq_num
        self.descriptor_type = args.descriptor

        self.gt_pairs = []
        self.poses = []
        self.keypoints = []
        self.scores = []
        self.descriptors = []
        self.dense_scans = []

        self.local_graph_range = [0, 0]
        self.divided_keypoints = np.array([])
        self.divided_dense_scans = []
        self.divided_seq_num = 0

        self._load_gt_pairs()
        self._load_datas()
        print("[Load] %d's poses SLAM data loaded" % len(self.poses))
    
    def _load_gt_pairs(self):
        file_path = os.path.join(self.dataset_dir, 'groundtruths128', '%02d'%self.seq, 'groundtruths.txt')
        with open(file_path, 'r') as f:
            lines_list = f.readlines()
            for i, line_str in enumerate(lines_list):
                if i == 0:
                    continue # skip the header line
                line_splitted = line_str.split()
                anc_idx = int(float(line_splitted[0]))
                pos_idx = int(float(line_splitted[1]))

                data = {'seq': self.seq, 'anc_idx': anc_idx, 'pos_idx': pos_idx}
                self.gt_pairs.append(data)

    def _load_datas(self):
        # poses
        pose_path = os.path.join(self.dataset_dir, 'poses', '%02d.txt'%self.seq)
        with open(pose_path, 'r') as f:
            for line in f.readlines():
                T_w_cam0 = np.fromstring(line, dtype=float, sep=' ')
                T_w_cam0 = T_w_cam0.reshape(3, 4)
                T_w_cam0 = np.vstack((T_w_cam0, [0, 0, 0, 1]))
                self.poses.append(T_w_cam0)

        # keypoints, scores and descriptors
        keypoint_folder = os.path.join(self.dataset_dir, 'keypoints', '%02d'%self.seq)
        keypoint_folder = os.listdir(keypoint_folder)   
        keypoint_folder.sort(key=lambda x:int(x[:-4]))
        for idx in range(len(keypoint_folder)):
            file = os.path.join(self.dataset_dir, 'keypoints', '%02d'%self.seq, keypoint_folder[idx])
            if os.path.isfile(file):
                pc = np.reshape(np.fromfile(file, dtype=np.float64), (-1, 139))
                self.keypoints.append(pc[:, :3])
                self.scores.append(pc[:, 3])
                self.descriptors.append(pc[:, 4:])
            else:
                self.keypoints.append([0.0, 0.0, 0.0])
                self.scores.append([0])
                self.descriptors.append([0.0]*135)

        # dense scans
        dense_folder = os.path.join(self.dataset_dir, 'dense_scan', '%02d'%self.seq)
        dense_folder = os.listdir(dense_folder)
        dense_folder.sort(key=lambda x:int(x[:-4]))
        for idx in range(len(dense_folder)):
            file = os.path.join(self.dataset_dir, 'dense_scan', '%02d'%self.seq, dense_folder[idx])
            if os.path.isfile(file):
                self.dense_scans.append(np.reshape(np.fromfile(file, dtype=np.float64), (-1, 3)))
            else:
                self.dense_scans.append(np.array([0, 0, 0]))

    def get_gt_pairs(self, idx):
        index_in_seq0 = self.gt_pairs[idx]['anc_idx']
        index_in_seq1 = self.gt_pairs[idx]['pos_idx']

        pose0 = torch.tensor(self.poses[index_in_seq0], dtype=torch.double)
        pose1 = torch.tensor(self.poses[index_in_seq1], dtype=torch.double)
        T_gt = torch.einsum('ab,de->ae', torch.inverse(pose0), pose1)

        pc1_w = self.dense_scans[index_in_seq0]
        pc2_w = self.dense_scans[index_in_seq1]

        pc1_w = np.array([(kp[0], kp[1], kp[2], 1) for kp in pc1_w]) 
        pc2_w = np.array([(kp[0], kp[1], kp[2], 1) for kp in pc2_w])
        pc1_w, pc2_w = torch.tensor(pc1_w, dtype=torch.double), torch.tensor(pc2_w, dtype=torch.double)

        kp0_np = np.array([(kp[0], kp[1], kp[2], 1) for kp in self.keypoints[index_in_seq0]]) 
        kp1_np = np.array([(kp[0], kp[1], kp[2], 1) for kp in self.keypoints[index_in_seq1]])
        kp0_tensor = torch.tensor(kp0_np, dtype=torch.double)
        kp1_tensor = torch.tensor(kp1_np, dtype=torch.double)
        pc1 = torch.einsum('ij,nj->ni', torch.inverse(pose0), pc1_w)
        pc2 = torch.einsum('ij,nj->ni', torch.inverse(pose1), pc2_w)
        kp0_local_tensor = torch.einsum('ij,nj->ni', torch.inverse(pose0), kp0_tensor).double()
        kp1_local_tensor = torch.einsum('ij,nj->ni', torch.inverse(pose1), kp1_tensor).double()

        kp0_local_tensor = kp0_local_tensor[:, :3]
        kp1_local_tensor = kp1_local_tensor[:, :3]
        pc1 = pc1[:, :3]
        pc2 = pc2[:, :3]

        desc0 = self.descriptors[index_in_seq0]
        desc1 = self.descriptors[index_in_seq1]

        kp0_num = len(kp0_tensor)
        kp1_num = len(kp1_tensor)

        norm0, norm1 = np.linalg.norm(desc0, axis=1), np.linalg.norm(desc1, axis=1)
        norm0, norm1 = norm0.reshape(kp0_num, 1), norm1.reshape(kp1_num, 1)
        epsilon = 1e-8  # small constant to prevent division by zero
        norm0, norm1 = norm0 + epsilon, norm1 + epsilon
        desc0, desc1 = np.where(norm0 != 0, np.multiply(desc0, 1/norm0), 0), np.where(norm1 != 0, np.multiply(desc1, 1/norm1), 0)

        desc0_tensor, desc1_tensor = torch.tensor(desc0, dtype=torch.double), torch.tensor(desc1, dtype=torch.double)
        scores0_tensor, scores1_tensor = torch.tensor(self.scores[index_in_seq0], dtype=torch.double), torch.tensor(self.scores[index_in_seq1], dtype=torch.double)

        dists = cdist(kp0_tensor, kp1_tensor)
        '''Find ground true keypoint matching'''
        min1 = np.argmin(dists, axis=0)
        min2 = np.argmin(dists, axis=1)
        min1v = np.min(dists, axis=1)
        min1f = min2[min1v < 0.5]

        '''For calculating repeatibility'''
        rep = len(min1f)

        match1, match2 = -1 * np.ones((len(kp0_tensor)), dtype=np.int16), -1 * np.ones((len(kp1_tensor)), dtype=np.int16)
        match1[min1v < 0.5] = min1f
        min2v = np.min(dists, axis=0)
        min2f = min1[min2v < 0.5]
        match2[min2v < 0.5] = min2f
            
        # print(kp0_tensor.shape)
        # print(kp0_tensor[:,:3].shape)

        return{
            # 'skip': False,
            'keypoints0': kp0_local_tensor[:,:3].unsqueeze(0),
            'keypoints1': kp1_local_tensor[:,:3].unsqueeze(0),
            'keypoints_global_0': kp0_tensor[:,:3].unsqueeze(0),
            'keypoints_global_1': kp1_tensor[:,:3].unsqueeze(0),
            'descriptors0': desc0_tensor.unsqueeze(0),
            'descriptors1': desc1_tensor.unsqueeze(0),
            'scores0': scores0_tensor.unsqueeze(0),
            'scores1': scores1_tensor.unsqueeze(0),
            'gt_matches0': match1,
            'gt_matches1': match2,
            'sequence': self.seq,
            'idx0': index_in_seq0,
            'idx1': index_in_seq1,
            'pose1': pose0,
            'pose2': pose1,
            # 'T_cam0_velo': T_cam0_velo,
            'T_gt': T_gt,
            'cloud0': pc1[:,:3].unsqueeze(0),
            'cloud1': pc2[:,:3].unsqueeze(0),
            # 'all_matches': list(all_matches),
            # 'file_name': file_name
            'rep': rep
        }
    
    def process_global_keypoints(self):
        pass
    
    def matching_test_1to2(self, idx, range_of_global_graph = 50, range_of_local_graph = 10):
        index_in_seq0 = self.gt_pairs[idx]['anc_idx']
        index_in_seq1 = self.gt_pairs[idx]['pos_idx']
        while index_in_seq0 <= range_of_global_graph:
            idx+=1
            index_in_seq0 = self.gt_pairs[idx]['anc_idx']
            index_in_seq1 = self.gt_pairs[idx]['pos_idx']
            
        pose0 = torch.tensor(self.poses[index_in_seq0], dtype=torch.double)
        pose1 = torch.tensor(self.poses[index_in_seq1], dtype=torch.double)
        T_gt = torch.einsum('ab,de->ae', torch.inverse(pose0), pose1)

        pc0_o3d = o3d.geometry.PointCloud()
        for i in range(index_in_seq0-range_of_global_graph, index_in_seq0):
            pc0_o3d.points.extend(self.dense_scans[i])
        pc0_o3d.voxel_down_sample(voxel_size=0.2)
        pc0 = np.array(pc0_o3d.points)
        pc1_o3d = o3d.geometry.PointCloud()
        for i in range(index_in_seq1, index_in_seq1+range_of_local_graph):
            pc1_o3d.points.extend(self.dense_scans[i])
        pc1_o3d.voxel_down_sample(voxel_size=0.2)
        pc1 = np.array(pc1_o3d.points)

        kp0_list = []
        kp0_for_desc = []
        desc0_list = []
        for i in range(index_in_seq0-range_of_global_graph, index_in_seq0):
            kp0_list.append(self.keypoints[i])
            for kp_idx in range(self.keypoints[i].shape[0]):
                kp0_for_desc.append(self.keypoints[i][kp_idx])
                desc0_list.append(self.descriptors[i][kp_idx])
        o3d_kp0 = o3d.geometry.PointCloud()
        for i in range(len(kp0_list)):
            o3d_kp0.points.extend(kp0_list[i])

        labels = np.array(o3d_kp0.cluster_dbscan(eps=0.3, min_points=4, print_progress=False))
        if len(labels) > 0:
            max_label = labels.max()  # max_label represents the number of clusters
            merged_keypoints_chach = [[] for _ in range(max_label+1)]
            
            # Group keypoints based on their labels (clusters)
            for idx, label in enumerate(labels):
                if label >= 0:
                    merged_keypoints_chach[label].append(np.array(o3d_kp0.points[idx]))
            keypoint_chach = []
            # Calculate the mean point for each cluster and add it to keypoint_chach
            for pi in merged_keypoints_chach:
                p = np.mean(pi, axis=0)
                keypoint_chach.append(p)
        kp0 = np.array(keypoint_chach)

        kp1_list = []
        kp1_for_desc = []
        desc1_list = []
        for i in range(index_in_seq1, index_in_seq1 + range_of_local_graph):
            kp1_list.append(self.keypoints[i])
            for kp_idx in range(self.keypoints[i].shape[0]):
                kp1_for_desc.append(self.keypoints[i][kp_idx])
                desc1_list.append(self.descriptors[i][kp_idx])
        o3d_kp1 = o3d.geometry.PointCloud()
        for i in range(len(kp1_list)):
            o3d_kp1.points.extend(kp1_list[i])

        labels = np.array(o3d_kp1.cluster_dbscan(eps=0.3, min_points=4, print_progress=False))
        if len(labels) > 0:
            max_label = labels.max()  # max_label represents the number of clusters
            merged_keypoints_chach = [[] for _ in range(max_label+1)]
            
            # Group keypoints based on their labels (clusters)
            for idx, label in enumerate(labels):
                if label >= 0:
                    merged_keypoints_chach[label].append(np.array(o3d_kp1.points[idx]))
            keypoint_chach = []
            # Calculate the mean point for each cluster and add it to keypoint_chach
            for pi in merged_keypoints_chach:
                p = np.mean(pi, axis=0)
                keypoint_chach.append(p)
        kp1 = np.array(keypoint_chach)

        kp0_np = np.array([(kp[0], kp[1], kp[2], 1) for kp in kp0])
        kp1_np = np.array([(kp[0], kp[1], kp[2], 1) for kp in kp1])
        kp0_tensor = torch.tensor(kp0_np, dtype=torch.double)
        kp1_tensor = torch.tensor(kp1_np, dtype=torch.double)
        kp0_num = len(kp0_tensor)
        kp1_num = len(kp1_tensor)

        desc0 = []
        scores0 = []
        desc1 = []
        scores1 = []
        for i in range(kp0.shape[0]):
            min_dist = 100000
            min_idx = 0
            for j in range(len(kp0_for_desc)):
                dist = np.linalg.norm(kp0[i] - kp0_for_desc[j])
                if dist < min_dist:
                    min_dist = dist
                    min_idx = j
            kp0[i] = kp0_for_desc[min_idx]
            desc0.append(desc0_list[min_idx])
            scores0.append(1)
        desc0 = np.array(desc0)
        scores0 = np.array(scores0)

        for i in range(kp1.shape[0]):
            min_dist = 100000
            min_idx = 0
            for j in range(len(kp1_for_desc)):
                dist = np.linalg.norm(kp1[i] - kp1_for_desc[j])
                if dist < min_dist:
                    min_dist = dist
                    min_idx = j
            kp1[i] = kp1_for_desc[min_idx]
            desc1.append(desc1_list[min_idx])
            scores1.append(1)
        desc1 = np.array(desc1)
        scores1 = np.array(scores1)

        pc0 = np.array([(kp[0], kp[1], kp[2], 1) for kp in pc0])
        pc1 = np.array([(kp[0], kp[1], kp[2], 1) for kp in pc1])
        pc0, pc1 = torch.tensor(pc0, dtype=torch.double), torch.tensor(pc1, dtype=torch.double)
        pc0 = torch.einsum('ij,nj->ni', torch.inverse(pose0), pc0)
        pc1 = torch.einsum('ij,nj->ni', torch.inverse(pose1), pc1)

        kp0_local_tensor = torch.einsum('ij,nj->ni', torch.inverse(pose0), kp0_tensor).double()
        kp1_local_tensor = torch.einsum('ij,nj->ni', torch.inverse(pose1), kp1_tensor).double()

        norm0, norm1 = np.linalg.norm(desc0, axis=1), np.linalg.norm(desc1, axis=1)
        norm0, norm1 = norm0.reshape(kp0_num, 1), norm1.reshape(kp1_num, 1)
        epsilon = 1e-8  # small constant to prevent division by zero
        norm0, norm1 = norm0 + epsilon, norm1 + epsilon
        desc0, desc1 = np.where(norm0 != 0, np.multiply(desc0, 1/norm0), 0), np.where(norm1 != 0, np.multiply(desc1, 1/norm1), 0)

        desc0_tensor, desc1_tensor = torch.tensor(desc0, dtype=torch.double), torch.tensor(desc1, dtype=torch.double)
        scores0_tensor, scores1_tensor = torch.tensor(scores0, dtype=torch.double), torch.tensor(scores1, dtype=torch.double)

        dists = cdist(kp0_tensor, kp1_tensor)
        '''Find ground true keypoint matching'''
        min1 = np.argmin(dists, axis=0)
        min2 = np.argmin(dists, axis=1)
        min1v = np.min(dists, axis=1)
        min1f = min2[min1v < 0.5]

        '''For calculating repeatibility'''
        rep = len(min1f)

        match1, match2 = -1 * np.ones((len(kp0_tensor)), dtype=np.int16), -1 * np.ones((len(kp1_tensor)), dtype=np.int16)
        match1[min1v < 0.5] = min1f
        min2v = np.min(dists, axis=0)
        min2f = min1[min2v < 0.5]
        match2[min2v < 0.5] = min2f

        return{
            # 'skip': False,
            'keypoints0': kp0_local_tensor[:,:3].unsqueeze(0),
            'keypoints1': kp1_local_tensor[:,:3].unsqueeze(0),
            'keypoints_global_0': kp0_tensor[:,:3].unsqueeze(0),
            'keypoints_global_1': kp1_tensor[:,:3].unsqueeze(0),
            'descriptors0': desc0_tensor.unsqueeze(0),
            'descriptors1': desc1_tensor.unsqueeze(0),
            'scores0': scores0_tensor.unsqueeze(0),
            'scores1': scores1_tensor.unsqueeze(0),
            'gt_matches0': match1,
            'gt_matches1': match2,
            'sequence': self.seq,
            'idx0': index_in_seq0,
            'idx1': index_in_seq1,
            'pose1': pose0,
            'pose2': pose1,
            # 'T_cam0_velo': T_cam0_velo,
            'T_gt': T_gt,
            'cloud0': pc0[:,:3].unsqueeze(0),
            'cloud1': pc1[:,:3].unsqueeze(0),
            # 'all_matches': list(all_matches),
            # 'file_name': file_name
            'rep': rep
        }
    
    def global_matching(self, idx):
        pass
                                     
    def __len__(self):
        return len(self.poses)

emplimentation 테스트 시나리오

In [4]:
# 1. 데이터 로드
dataset = Kitti_Rops_dataset_loader(opt, 8)

[Load] 1396's poses SLAM data loaded


In [7]:
# 2. 모델 로드
from models.mdgat_denseKITTI import MDGAT
path_checkpoint = opt.resume_model
checkpoint = torch.load(path_checkpoint, map_location={'cuda:2':'cuda:0'})
lr = checkpoint['lr_schedule']
config = {
        'net': {
            'sinkhorn_iterations': opt.sinkhorn_iterations,
            'match_threshold': opt.match_threshold,
            'lr': lr,
            'loss_method': opt.loss_method,
            'k': opt.k,
            'descriptor': opt.descriptor,
            'mutual_check': opt.mutual_check,
            'triplet_loss_gamma': opt.triplet_loss_gamma,
            'train_step':opt.train_step,
            'L':opt.l,
            'scheduler_gamma': 0.1**(1/100),
            'descriptor_dim': 128,
            'keypoint_encoder': [32, 64, 128],
            'descritor_encoder': [64, 128],
            'GNN_layers': ['self', 'cross'] * 9,
        }
    }
net = MDGAT(config.get('net', {}))
optimizer = torch.optim.Adam(net.parameters(), lr=config.get('net', {}).get('lr'))
net = torch.nn.DataParallel(net)
net.load_state_dict(checkpoint['net'])
net.double().eval()

if torch.cuda.is_available():
    # torch.cuda.set_device(opt.local_rank)
    device=torch.device('cuda:{}'.format(opt.local_rank))
else:
    device = torch.device("cpu")
    print("### CUDA not available ###")

net.to(device)

DataParallel(
  (module): MDGAT(
    (penc): PointnetEncoder(
      (sa1): PointNetSetKptsMsg(
        (conv_blocks): ModuleList(
          (0): ModuleList(
            (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (bn_blocks): ModuleList(
          (0): ModuleList(
            (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
      )
      (sa2): PointNetSetAbstraction(
        (mlp_convs): ModuleList(
          (0): Conv2d(131, 256, kernel_size=(1, 1), stride=(1, 1))
          (1): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
          (2): Conv2d(256, 128, ke

매트릭 코드 짜보기

In [8]:
mean_test_loss = []; precision_array = []; accuracy_array = []; recall_array = []
trans_error_array = []; rot_error_array = []; relative_trans_error_array = []; relative_rot_error_array = []
repeatibilty_array = []; valid_num_array = []; all_num_array = []; inlier_array = [] 
kpnum_array = []; fp_rate_array = []; tp_rate_array = []; tp_rate2_array = []; inlier_ratio_array= [];tm_a=[];fm_a=[]
fail = 0
baned_data = 0
pair_set = tqdm(range(len(dataset.gt_pairs))) 
for pair in pair_set:
# for pair in [0]:
    pred = dataset.get_gt_pairs(pair)
    for p in pred:
        if type(pred[p]) == torch.Tensor:
            pred[p] = pred[p].to(device)
    # pred['descriptors1'] = net.module.infer_desc(pred['keypoints1'], pred['cloud1'])
    # print(pred['descriptors1'].shape)
    # pred['descriptors0'] = net.module.infer_desc(pred['keypoints0'], pred['cloud0'])
    # print(pred['descriptors0'].shape)
    
    # print(pred['cloud0'].shape, pred['cloud1'].shape)
    data = net.module.infer(pred)
    # data = net(pred)
    pred = {**pred, **data}

    kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
    kpts_g_0, kpts_g_1 = pred['keypoints_global_0'][0].cpu().numpy(), pred['keypoints_global_1'][0].cpu().numpy()
    matches0, matches1, conf = pred['matches0'][0].cpu().detach().numpy(), pred['matches1'][0].cpu().detach().numpy(), pred['matching_scores0'][0].cpu().detach().numpy()
    gt_match0, gt_match1 = pred['gt_matches0'], pred['gt_matches1']
    valid = matches0 > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[matches0[valid]]

    mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
    mutual0 = np.arange(len(matches0))[valid][mutual0]
    mutual1 = matches0[mutual0]
    x = np.ones(len(matches1)) == 1
    x[mutual1] = False
    valid1 = matches1 > -1

    mconf = conf[valid]

    ## ground truth ##
    matches_gt, matches_gt1 = pred['gt_matches0'], pred['gt_matches1']
    matches_gt[matches_gt == len(matches_gt1)] = -1
    matches_gt1[matches_gt1 == len(matches_gt)] = -1
    valid_gt = matches_gt > -1

    valid_num = np.sum(valid_gt)
    all_num = len(valid_gt)
    repeatibilty = valid_num/all_num
    repeatibilty_array.append(repeatibilty)

    mkpts0_gt = kpts0[valid_gt]
    mkpts1_gt = kpts1[matches_gt[valid_gt]]
    mutual0 = np.arange(len(matches_gt))[valid_gt] == matches_gt1[matches_gt[valid_gt]]
    # mutual0_inv = 1-mutual0
    mutual0 = np.arange(len(matches_gt))[valid_gt][mutual0]
    mutual1 = matches_gt[mutual0]
    x = np.ones(len(matches_gt1)) == 1
    x[mutual1] = False               
    valid_gt1 = matches_gt1 > -1

    mscores_gt = pred['scores0'][0].cpu().numpy()[valid_gt]
    gt_idx = np.arange(len(kpts0))[valid_gt]

    if len(mkpts0) < 4:
        fail+=1
        print('registration fail')

    ''' calculate false positive ,true positive ,true nagetive, precision, accuracy, recall '''
    true_positive = [(matches0[i] == matches_gt[i]) and (valid[i]) for i in range(len(kpts0))]
    true_negativate = [(matches0[i] == matches_gt[i]) and not (valid[i]) for i in range(len(kpts0))]
    false_positive = [valid[i] and (matches_gt[i]==-1) for i in range(len(kpts0))]
    ckpts0 = kpts0[true_positive]
    ckpts1 = [matches0[true_positive]]
    precision = np.sum(true_positive) / np.sum(valid) if np.sum(valid) > 0 else 0
    recall = np.sum(true_positive) / np.sum(valid_gt) if np.sum(valid) > 0 else 0
    tm = np.sum(true_positive) 
    fm = np.sum(false_positive) 
    matching_score = np.sum(true_positive) / len(kpts0) if len(kpts0) > 0 else 0
    accuracy = (np.sum(true_positive) + np.sum(true_negativate))/len(matches_gt)
    fp_rate = np.sum(false_positive)/np.sum(matches_gt==-1)
    tp_rate = np.sum([valid[i] and (matches_gt[i]>-1) for i in range(len(kpts0))])/np.sum(matches_gt > -1)
    tp_rate2 = np.sum(true_positive)/np.sum(matches_gt > -1)
    T=[]
    # print('idx{}, precision {:.3f}, accuracy {:.3f}, recall {:.3f}, true match {:.3f}, false match {:.3f}, fp_rate {:.3f}, tp_rate {:.3f}'.format(
    #     pair, precision, accuracy, recall,tm,fm, fp_rate, tp_rate))
    precision_array.append(precision)
    accuracy_array.append(accuracy)
    recall_array.append(recall)
    fp_rate_array.append(fp_rate)
    tp_rate_array.append(tp_rate)
    tp_rate2_array.append(tp_rate2)
    tm_a.append(tm)
    fm_a.append(fm)
precision_mean = np.mean(precision_array)
accuracy_mean = np.mean(accuracy_array)
recall_mean = np.mean(recall_array)
repeatibilty_array_mean = np.mean(repeatibilty_array)
fp_rate_mean = np.mean(fp_rate_array)
tp_rate_mean = np.mean(tp_rate_array)
tp_rate_mean2 = np.mean(tp_rate2_array)
tm = np.mean(tm_a)
fm = np.mean(fm_a)

print('precision {:.3f}, accuracy {:.3f}, recall {:.3f}, true match {:.3f}, false match {:.3f}, fp_rate {:.3f}, tp_rate {:.3f}'.format(
        precision_mean, accuracy_mean, recall_mean,tm,fm, fp_rate_mean, tp_rate_mean))
# print('average repeatibility: {:.3f}, fail {:.6f}, precision_mean {:.3f}, accuracy_mean {:.3f}, recall_mean {:.3f}, true match {:.3f}, false match {:.3f}, fp_rate_mean {:.3f}, tp_rate_mean {:.3f}, tp_rate_mean2 {:.3f}, trans_error_mean {:.3f}, rot_error_mean {:.3f} '.format(
#     repeatibilty_array_mean, fail/pair, precision_mean, accuracy_mean, recall_mean,tm,fm, fp_rate_mean, tp_rate_mean, tp_rate_mean2, trans_error_mean, rot_error_mean ))
# print('valid num {}, all_num {}'.format(valid_num_mean, all_num_mean))
print('baned_data {}'.format(baned_data))

#average repeatibility: 0.593, fail 0.002518, precision_mean 0.947, accuracy_mean 0.944, recall_mean 0.915, true match 111.825, false match 2.146, fp_rate_mean 0.027, tp_rate_mean 0.939, tp_rate_mean2 0.915, trans_error_mean nan, rot_error_mean nan 
# baned_data 0.0
                

100%|██████████| 3972/3972 [12:24<00:00,  5.34it/s]

precision 0.727, accuracy 0.774, recall 0.752, true match 94.036, false match 19.574, fp_rate 0.254, tp_rate 0.837
baned_data 0


매칭 비주얼로 보기

In [8]:
for pair in range(0, len(dataset.gt_pairs), 10):
    pred = dataset.get_gt_pairs(pair)
    for p in pred:
        if type(pred[p]) == torch.Tensor:
            pred[p] = pred[p].to(device)
    data = net.module.infer(pred)
    pred = {**pred, **data}

    kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
    kpts_g_0, kpts_g_1 = pred['keypoints_global_0'][0].cpu().numpy(), pred['keypoints_global_1'][0].cpu().numpy()
    matches0, matches1, conf = pred['matches0'][0].cpu().detach().numpy(), pred['matches1'][0].cpu().detach().numpy(), pred['matching_scores0'][0].cpu().detach().numpy()
    gt_match0, gt_match1 = pred['gt_matches0'], pred['gt_matches1']
    valid = matches0 > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[matches0[valid]]

    mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
    mutual0 = np.arange(len(matches0))[valid][mutual0]
    mutual1 = matches0[mutual0]
    x = np.ones(len(matches1)) == 1
    x[mutual1] = False
    valid1 = matches1 > -1

    mconf = conf[valid]    

    pcd_kp0 = o3d.geometry.PointCloud()
    pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
    pcd_kp0.paint_uniform_color([1, 0, 0])
    pcd_kp1 = o3d.geometry.PointCloud()
    pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
    pcd_kp1.paint_uniform_color([0, 1, 0])

    points = np.concatenate((np.array(pcd_kp0.points),np.array(pcd_kp1.points)), axis=0) # >> pcd_kp0에 pcd_kp1를 이어 붙힘
    lines = []
    colors = []
    for idx, match in enumerate(mutual0): # mutual0의 값
        lines.append([match, mutual1[idx] + len(kpts0)])
        # lines.append([match, mutual1[idx] + len(kpts_g_0)])
        point1 = kpts_g_0[match]
        point2 = kpts_g_1[mutual1[idx]]
        if np.linalg.norm(point1 - point2) < 1.0:
            colors.append([0, 1, 0])
        else: 
            colors.append([1, 0, 0])
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector((points)),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    # o3d.visualization.draw_geometries([pcd0,pcd1,line_set])

    # o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])
    o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])

  

KeyboardInterrupt: 

실제상황처럼 테스트 해보기.

In [9]:
# for pair in range(160, len(dataset.gt_pairs), 10):
pair = 140

for range0 in [10, 20, 30, 40, 50, 60]:
    for range1 in [10, 20, 30, 40, 50, 60]:
        pred = dataset.matching_test_1to2(pair, range0, range1)
        for p in pred:
            if type(pred[p]) == torch.Tensor:
                pred[p] = pred[p].to(device)
        print(pred['keypoints0'].shape, pred['keypoints1'].shape, pred['descriptors0'].shape, pred['descriptors1'].shape, pred['scores0'].shape, pred['scores1'].shape)

        data = net.module.infer(pred)
        pred = {**pred, **data}

        kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
        kpts_g_0, kpts_g_1 = pred['keypoints_global_0'][0].cpu().numpy(), pred['keypoints_global_1'][0].cpu().numpy()
        matches0, matches1, conf = pred['matches0'][0].cpu().detach().numpy(), pred['matches1'][0].cpu().detach().numpy(), pred['matching_scores0'][0].cpu().detach().numpy()
        gt_match0, gt_match1 = pred['gt_matches0'], pred['gt_matches1']
        valid = matches0 > -1
        mkpts0 = kpts0[valid]
        mkpts1 = kpts1[matches0[valid]]

        mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
        mutual0 = np.arange(len(matches0))[valid][mutual0]
        mutual1 = matches0[mutual0]
        x = np.ones(len(matches1)) == 1
        x[mutual1] = False
        valid1 = matches1 > -1

        mconf = conf[valid]    

        pcd_kp0 = o3d.geometry.PointCloud()
        pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
        pcd_kp0.paint_uniform_color([1, 0, 0])
        pcd_kp1 = o3d.geometry.PointCloud()
        pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
        pcd_kp1.paint_uniform_color([0, 1, 0])

        points = np.concatenate((np.array(pcd_kp0.points),np.array(pcd_kp1.points)), axis=0) # >> pcd_kp0에 pcd_kp1를 이어 붙힘
        lines = []
        colors = []
        for idx, match in enumerate(mutual0): # mutual0의 값
            lines.append([match, mutual1[idx] + len(kpts0)])
            # lines.append([match, mutual1[idx] + len(kpts_g_0)])
            point1 = kpts_g_0[match]
            point2 = kpts_g_1[mutual1[idx]]
            if np.linalg.norm(point1 - point2) < 1.0:
                colors.append([0, 1, 0])
            else: 
                colors.append([1, 0, 0])
        line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector((points)),
            lines=o3d.utility.Vector2iVector(lines),
        )
        line_set.colors = o3d.utility.Vector3dVector(colors)
        # o3d.visualization.draw_geometries([pcd0,pcd1,line_set])

        # o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])
        o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])

torch.Size([1, 184, 3]) torch.Size([1, 179, 3]) torch.Size([1, 184, 135]) torch.Size([1, 179, 135]) torch.Size([1, 184]) torch.Size([1, 179])
torch.Size([1, 184, 3]) torch.Size([1, 298, 3]) torch.Size([1, 184, 135]) torch.Size([1, 298, 135]) torch.Size([1, 184]) torch.Size([1, 298])
torch.Size([1, 184, 3]) torch.Size([1, 415, 3]) torch.Size([1, 184, 135]) torch.Size([1, 415, 135]) torch.Size([1, 184]) torch.Size([1, 415])


OutOfMemoryError: CUDA out of memory. Tried to allocate 6.58 GiB (GPU 0; 23.64 GiB total capacity; 20.27 GiB already allocated; 1.38 GiB free; 20.59 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# for pair in range(160, len(dataset.gt_pairs), 10):
pair = 140
for range0 in [10, 20, 30, 40, 50, 60]:
    for range1 in [10, 20, 30, 40, 50, 60]:
        pred = dataset.matching_test_1to2(pair, range0, range1)
        for p in pred:
            if type(pred[p]) == torch.Tensor:
                pred[p] = pred[p].to(device)
        print(pred['keypoints0'].shape, pred['keypoints1'].shape, pred['descriptors0'].shape, pred['descriptors1'].shape, pred['scores0'].shape, pred['scores1'].shape)

        data = net.module.infer_mdgat(pred)
        pred = {**pred, **data}

        kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
        kpts_g_0, kpts_g_1 = pred['keypoints_global_0'][0].cpu().numpy(), pred['keypoints_global_1'][0].cpu().numpy()
        matches0, matches1, conf = pred['matches0'][0].cpu().detach().numpy(), pred['matches1'][0].cpu().detach().numpy(), pred['matching_scores0'][0].cpu().detach().numpy()
        gt_match0, gt_match1 = pred['gt_matches0'], pred['gt_matches1']
        valid = matches0 > -1
        mkpts0 = kpts0[valid]
        mkpts1 = kpts1[matches0[valid]]

        mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
        mutual0 = np.arange(len(matches0))[valid][mutual0]
        mutual1 = matches0[mutual0]
        x = np.ones(len(matches1)) == 1
        x[mutual1] = False
        valid1 = matches1 > -1

        mconf = conf[valid]

        gt_match_num = 0
        for i in gt_match0:
            if i > -1:
                gt_match_num+=1
                
        matched_num = 0
        miss_matched_num = 0
        for idx, match in enumerate(mutual0): # mutual0의 값
            point1 = kpts_g_0[match]
            point2 = kpts_g_1[mutual1[idx]]
            if np.linalg.norm(point1 - point2) < 0.5:
                matched_num+=1
            else: 
                miss_matched_num+=1

        print('range0 {}, range1 {}, gt_match_num {}, matched_num {}, miss_matched_num {}'.format(range0, range1, gt_match_num, matched_num, miss_matched_num))

In [ ]:
kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
kpts_g_0, kpts_g_1 = pred['keypoints_global_0'][0].cpu().numpy(), pred['keypoints_global_1'][0].cpu().numpy()
matches0, matches1, conf = pred['matches0'][0].cpu().detach().numpy(), pred['matches1'][0].cpu().detach().numpy(), pred['matching_scores0'][0].cpu().detach().numpy()
gt_match0, gt_match1 = pred['gt_matches0'], pred['gt_matches1']
valid = matches0 > -1
mkpts0 = kpts0[valid]
mkpts1 = kpts1[matches0[valid]]

mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
mutual0 = np.arange(len(matches0))[valid][mutual0]
mutual1 = matches0[mutual0]
x = np.ones(len(matches1)) == 1
x[mutual1] = False
valid1 = matches1 > -1

mconf = conf[valid]

# matches_gt, matches_gt1 = pred['gt_matches0'], pred['gt_matches1']
# matches_gt[matches_gt == len(matches_gt1)] = -1
# matches_gt1[matches_gt1 == len(matches_gt)] = -1
# valid_gt = matches_gt > -1

In [ ]:
print(mutual0)

In [ ]:
pcd_kp_g_0 = o3d.geometry.PointCloud()
pcd_kp_g_0.points = o3d.utility.Vector3dVector(kpts_g_0)
pcd_kp_g_0.paint_uniform_color([0, 0, 1])
pcd_kp_g_1 = o3d.geometry.PointCloud()
pcd_kp_g_1.points = o3d.utility.Vector3dVector(kpts_g_1)
pcd_kp_g_1.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([pcd_kp_g_0, pcd_kp_g_1, line_set])

In [ ]:
# 시각화
pcd_kp0 = o3d.geometry.PointCloud()
pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
pcd_kp0.paint_uniform_color([1, 0, 0])
pcd_kp1 = o3d.geometry.PointCloud()
pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
pcd_kp1.paint_uniform_color([0, 1, 0])
pcd_kp_g_0 = o3d.geometry.PointCloud()
pcd_kp_g_0.points = o3d.utility.Vector3dVector(kpts_g_0)
pcd_kp_g_0.paint_uniform_color([0, 0, 1])
pcd_kp_g_1 = o3d.geometry.PointCloud()
pcd_kp_g_1.points = o3d.utility.Vector3dVector(kpts_g_1)
pcd_kp_g_1.paint_uniform_color([0, 1, 0])

points = np.concatenate((np.array(pcd_kp_g_0.points),np.array(pcd_kp_g_1.points)), axis=0) # >> pcd_kp0에 pcd_kp1를 이어 붙힘
lines = []
colors = []
for idx, match in enumerate(mutual0): # mutual0의 값
    lines.append([match, mutual1[idx] + len(kpts0)])
    # lines.append([match, mutual1[idx] + len(kpts_g_0)])
    point1 = kpts_g_0[match]
    point2 = kpts_g_1[mutual1[idx]]
    if np.linalg.norm(point1 - point2) < 1.0:
        colors.append([0, 1, 0])
    else: 
        colors.append([1, 0, 0])
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector((points)),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
# o3d.visualization.draw_geometries([pcd0,pcd1,line_set])

# o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])
o3d.visualization.draw_geometries([pcd_kp_g_0, pcd_kp_g_1, line_set])

In [ ]:
print(kpts0.shape, kpts_g_0.shape)

In [ ]:
pred = dataset.get_divided_data(2)
print(pred['keypoints0'].shape, pred['cloud0'].shape, pred['scores0'].shape)    

In [ ]:
# 매칭
# pred = dataset.get_matching_data(12)
# pred = dataset.get_matching_data(13)
# pred = dataset.get_matching_data(12)
for p in pred:
    if type(pred[p]) == torch.Tensor:
        pred[p] = pred[p].to(device)
print(pred['keypoints0'].shape, pred['keypoints1'].shape, pred['descriptors0'].shape, pred['descriptors1'].shape)

data = net.module.infer_mdgat(pred, [pred['keypoints0'].shape[1]//2, None, pred['keypoints0'].shape[1]//2, None, pred['keypoints0'].shape[1]//4, None, pred['keypoints0'].shape[1]//4, None], [pred['keypoints1'].shape[1]//2, None, pred['keypoints1'].shape[1]//2, None, pred['keypoints1'].shape[1]//4, None, pred['keypoints1'].shape[1]//4, None])
pred = {**pred, **data}
print("pred's keys: ", pred.keys())

In [ ]:
# 3. 디스크립터 추출
net.double().eval()

for i in range(dataset.divided_seq_num - 1):
    pred = dataset.get_divided_data(i)
    for p in pred:
        pred[p] = pred[p].to(device)
    data = net.module.infer_desc(pred)
    for d in data:
        data[d] = data[d].detach().cpu()
    dataset.push_descriptor(i, data['desc'])

In [ ]:
# 4. 매칭
# pred = dataset.get_matching_data(12)
pred = dataset.get_matching_data(13)
# pred = dataset.get_matching_data(12)
for p in pred:
    if type(pred[p]) == torch.Tensor:
        pred[p] = pred[p].to(device)
print(pred['keypoints0'].shape, pred['keypoints1'].shape, pred['descriptors0'].shape, pred['descriptors1'].shape)

data = net.module.infer_mdgat(pred, [pred['keypoints0'].shape[1]//2, None, pred['keypoints0'].shape[1]//2, None, pred['keypoints0'].shape[1]//4, None, pred['keypoints0'].shape[1]//4, None], [pred['keypoints1'].shape[1]//2, None, pred['keypoints1'].shape[1]//2, None, pred['keypoints1'].shape[1]//4, None, pred['keypoints1'].shape[1]//4, None])
pred = {**pred, **data}
print("pred's keys: ", pred.keys())

In [ ]:
kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
matches0, matches1, conf = pred['matches0'][0].cpu().detach().numpy(), pred['matches1'][0].cpu().detach().numpy(), pred['matching_scores0'][0].cpu().detach().numpy()
valid = matches0 > -1
mkpts0 = kpts0[valid]
mkpts1 = kpts1[matches0[valid]]

mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
mutual0 = np.arange(len(matches0))[valid][mutual0]
mutual1 = matches0[mutual0]
x = np.ones(len(matches1)) == 1
x[mutual1] = False
valid1 = matches1 > -1

mconf = conf[valid]

matches_gt, matches_gt1 = pred['gt_matches0'], pred['gt_matches1']
matches_gt[matches_gt == len(matches_gt1)] = -1
matches_gt1[matches_gt1 == len(matches_gt)] = -1
valid_gt = matches_gt > -1

print(matches0.shape, matches1.shape, conf.shape) # >> (3328,) (256,) (3328,)

print(matches0[669]) # matches0의 669번째 인덱스와의 매칭 결과 >> 1

print(matches1[:]) # matches1의 전체 결과 >> [  -1  669   -1  677 ...], 즉 matches1의 1번째 포인트가 matches0의 669번째 포인트와 매칭됨

print(mutual0) # >> [   9   10   13   15 ...]

print(mutual1) # >> [248 245  20 255 106 ...]

print(matches0[9], matches1[248]) # >> 248, 9, 즉 mutual0[i]와 mutual1[i]의 idx끼리 매칭 됨

In [ ]:
# 시각화
pcd_kp0 = o3d.geometry.PointCloud()
pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
pcd_kp0.paint_uniform_color([1, 0, 0])
pcd_kp1 = o3d.geometry.PointCloud()
pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
pcd_kp1.paint_uniform_color([0, 1, 0])

points = np.concatenate((np.array(pcd_kp0.points),np.array(pcd_kp1.points)), axis=0) # >> pcd_kp0에 pcd_kp1를 이어 붙힘
lines = []
for idx, match in enumerate(mutual0): # mutual0의 값
    # lines.append([match, match + 1])
    # lines.append([mutual1[idx] + len(kpts0), mutual1[idx] + len(kpts0)])
    lines.append([match, mutual1[idx] + len(kpts0)])
colors = [[0, 1, 0] for _ in range(len(lines))] # lines are shown in green
# print(points[lines[0][0]], points[lines[0][1]])
# print(pcd_kp0.points[lines[0][0]], pcd_kp1.points[lines[0][1] - len(pcd_kp0.points)])
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector((points)),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
# o3d.visualization.draw_geometries([pcd0,pcd1,line_set])

o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])

In [ ]:
ttt = 2.0

# 시각화
kpts0, kpts1 = pred['keypoints0'][0].cpu().numpy(), pred['keypoints1'][0].cpu().numpy()
pcd_kp0 = o3d.geometry.PointCloud()
pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
pcd_kp0.paint_uniform_color([1, 0, 0])
pcd_kp1 = o3d.geometry.PointCloud()
pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
pcd_kp1.paint_uniform_color([0, 1, 0])

desc0 = pred['descriptors0'][0].cpu().detach().numpy().T
desc1 = pred['descriptors1'][0].cpu().detach().numpy().T
dists = cdist(desc0, desc1)
min0 = np.argmin(dists, axis=0)
min1 = np.argmin(dists, axis=1)
min0v = np.min(dists, axis=1)
min0f = min1[min0v < ttt]
match0, match1 = -1 * np.ones((len(desc0)), dtype=np.int16), -1 * np.ones((len(desc1)), dtype=np.int16)
match0[min0v < ttt] = min0f
min1v = np.min(dists, axis=0)
min1f = min0[min1v < ttt]
match1[min1v < ttt] = min1f
mutual0 = np.arange(len(matches0))[valid] == matches1[matches0[valid]]
mutual0 = np.arange(len(matches0))[valid][mutual0]
mutual1 = matches0[mutual0]

# 시각화
pcd_kp0 = o3d.geometry.PointCloud()
pcd_kp0.points = o3d.utility.Vector3dVector(kpts0)
pcd_kp0.paint_uniform_color([1, 0, 0])
pcd_kp1 = o3d.geometry.PointCloud()
pcd_kp1.points = o3d.utility.Vector3dVector(kpts1)
pcd_kp1.paint_uniform_color([0, 1, 0])

points = np.concatenate((np.array(pcd_kp0.points),np.array(pcd_kp1.points)), axis=0) # >> pcd_kp0에 pcd_kp1를 이어 붙힘
lines = []
for idx, match in enumerate(mutual0): # mutual0의 값
    lines.append([match, mutual1[idx] + len(kpts0)])
colors = [[0, 1, 0] for _ in range(len(lines))] # lines are shown in green
# print(points[lines[0][0]], points[lines[0][1]])
# print(pcd_kp0.points[lines[0][0]], pcd_kp1.points[lines[0][1] - len(pcd_kp0.points)])
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector((points)),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
# o3d.visualization.draw_geometries([pcd0,pcd1,line_set])

o3d.visualization.draw_geometries([pcd_kp0, pcd_kp1, line_set])


In [ ]:
print(mutual0)
print(mutual1)